<h1> Autoencoder </h1>
Autoencoders are a type of unsupervised learning that can be used to compress input data into a lower-dimensional encoding. The objective of training an Auto-Encoder is to create an Encoder that can represent data in a lower-dimensional space, and a Decoder that can take us from this lower-dimensional "latent" space back to the original data space.
<img src="https://upload.wikimedia.org/wikipedia/commons/2/28/Autoencoder_structure.png" width="500" align="center">

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets as Datasets
import torchvision.transforms as transforms
import torch.nn.functional as F
import torchvision.models as models
import torchvision.utils as vutils

import os
import random
import numpy as np
import math
from IPython.display import clear_output
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.notebook import trange, tqdm

In [ ]:
batchSize = 64

# Define learning rate
lr = 1e-4

# Number of Training epochs
nepoch = 10

# The size of the Latent Vector
latent_size = 128

root = "../../datasets"

noise_scale = 0.3

In [ ]:
use_cuda = torch.cuda.is_available()
gpu_indx  = 0
device = torch.device(gpu_indx if use_cuda else "cpu")

<h3>Create an MNIST dataset and dataloader</h3>

In [ ]:
# Define our transform
# We'll upsample the images to 32x32 as it's easier to contruct our network
transform = transforms.Compose([
            transforms.Resize(32),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5])])

train_set = Datasets.MNIST(root=root, train=True, transform=transform, download=True)
train_loader = DataLoader(train_set, batch_size=batchSize,shuffle=True, num_workers=4)

test_set = Datasets.MNIST(root=root, train=False, transform=transform, download=True)
test_loader = DataLoader(test_set, batch_size=batchSize, shuffle=False, num_workers=4)

## AE Network

In [ ]:
# We split up our network into two parts, the Encoder and the Decoder
class Encoder(nn.Module):
    def __init__(self, channels, ch=32, z=32):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv2d(channels, ch, 4, 2, 1)
        self.bn1 = nn.BatchNorm2d(ch)
        self.conv2 = nn.Conv2d(ch, 2*ch, 4, 2, 1)
        self.bn2 = nn.BatchNorm2d(2*ch)
        self.conv3 = nn.Conv2d(2*ch, 4*ch, 4, 2, 1)
        self.bn3 = nn.BatchNorm2d(4*ch)

        self.conv_out = nn.Conv2d(4*ch, z, 4, 1)
        
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))

        return self.conv_out(x)
    
class Decoder(nn.Module):
    def __init__(self, channels, ch = 32, z = 32):
        super(Decoder, self).__init__()
        
        self.conv1 = nn.ConvTranspose2d(z, 4*ch, 4, 1)
        self.bn1 = nn.BatchNorm2d(4*ch)
        self.conv2 = nn.ConvTranspose2d(4*ch, 2*ch, 4, 2, 1)
        self.bn2 = nn.BatchNorm2d(2*ch)
        self.conv3 = nn.ConvTranspose2d(2*ch, ch, 4, 2, 1)
        self.bn3 = nn.BatchNorm2d(ch)
        self.conv4 = nn.ConvTranspose2d(ch, channels, 4, 2, 1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.conv4(x)

        return torch.tanh(x)
    
class AE(nn.Module):
    def __init__(self, channel_in, ch=16, z=32):
        super(AE, self).__init__()
        self.encoder = Encoder(channels=channel_in, ch=ch, z=z)
        self.decoder = Decoder(channels=channel_in, ch=ch, z=z)

    def forward(self, x):
        encoding = self.encoder(x)
        x = self.decoder(encoding)
        return x, encoding

<h3>Visualize our data</h3>

In [ ]:
# Get a test image
dataiter = iter(test_loader)
test_images = dataiter.next()[0]
# View the shape
test_images.shape

In [ ]:
# Visualize the data!!!
plt.figure(figsize = (20,10))
out = vutils.make_grid(test_images[0:8], normalize=True)
_ = plt.imshow(out.numpy().transpose((1, 2, 0)))

### De-noising Autoencoder
Lets use the auto encoder architecture to remove some noise from an image!

In [ ]:
# Visualize the data!!!
plt.figure(figsize = (20, 10))
random_sample = (torch.bernoulli((1 - noise_scale) * torch.ones_like(test_images)) * 2) - 1
noisy_test_img = random_sample * test_images

out = vutils.make_grid(noisy_test_img[0:8], normalize=True)
_ = plt.imshow(out.numpy().transpose((1, 2, 0)))

<h3>Create Network and Optimizer</h3>

In [ ]:
# Create our network
ae_net = AE(channel_in=1, z=latent_size).to(device)
# Setup optimizer
optimizer = optim.Adam(ae_net.parameters(), lr=lr)

loss_func = nn.MSELoss()

loss_log = []
train_loss = 0

<h4>Network output</h4>

In [ ]:
# Pass through a test image to make sure everything is working
recon_data, encoding = ae_net(test_images.to(device))

# View the Latent vector shape
encoding.shape

<h2>Start training!</h2>

In [ ]:
pbar = trange(0, nepoch, leave=False, desc="Epoch")    
for epoch in pbar:
    pbar.set_postfix_str('Loss: %.4f' % train_loss)
    for i, data in enumerate(tqdm(train_loader, leave=False, desc="Training")):

        image = data[0].to(device)
        
        random_sample = (torch.bernoulli((1 - noise_scale) * torch.ones_like(image)) * 2) - 1
        noisy_img = random_sample * image
        
        # Forward pass the image in the data tuple
        recon_data, _ = ae_net(noisy_img)
        
        # Calculate the MSE loss
        loss = loss_func(recon_data, image)
        
        # Log the loss
        loss_log.append(loss.item())
        train_loss = loss.item()
        
        # Take a training step
        ae_net.zero_grad()
        loss.backward()
        optimizer.step()

## Results!

In [ ]:
_ = plt.plot(loss_log)
_ = plt.title("MSE Loss")

In [ ]:
# Ground Truth
plt.figure(figsize = (20,10))
out = vutils.make_grid(test_images[0:8], normalize=True)
_ = plt.imshow(out.numpy().transpose((1, 2, 0)))

In [ ]:
# Noisy Input
plt.figure(figsize = (20,10))
out = vutils.make_grid(noisy_test_img[0:8], normalize=True)
_ = plt.imshow(out.numpy().transpose((1, 2, 0)))

In [ ]:
# Reconstruction
plt.figure(figsize = (20,10))
recon_data, _ = ae_net(noisy_test_img.to(device))
out = vutils.make_grid(recon_data.detach().cpu()[0:8], normalize=True)
_ = plt.imshow(out.numpy().transpose((1, 2, 0)))